In [ ]:
import pandas as pd #데이터 전처리

#데이터셋 읽어오기
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
#gender_submission=pd.read_csv('../input/titanic/gender_submission.csv')

#train set 확인
train.head()

In [ ]:
train.info() #Non-Null Count가 다름 => 중간중간 Nan 값 존재
train.isnull().sum() #결측값 존재하는 거 확인

In [ ]:
#Embarked,Fare 결측값 채우기

train['Embarked'].value_counts() #S값이 가장 많음
train['Embarked'].fillna('S',inplace=True) # S로 채워넣음

#결측값- 평균값으로 채우기
test['Fare'].fillna(test['Fare'].mean(),inplace=True)

In [ ]:
#결측값 처리 - 호칭에 따라 구분

train['identity']=train.Name.str.extract(' ([A-Za-z]+)\.')
test['identity']=train.Name.str.extract(' ([A-Za-z]+)\.')
# ' ([A-Za-z]+).' 는 공백으로 시작하고, .으로 끝나는 문자열을 추출할 때 사용

pd.crosstab(train['identity'],train['Sex'])
#범주형 변수를 기준으로 개수 파악

#Ms Mr 등 인원수가 많은 호칭 제외하고 나머지는 other로 분류
train['identity']=train['identity'].replace(['Capt','Col','Countess','Don','Dr','Jonkheer','Lady','Major','Mlle','Master','Mile','Mme','Ms','Rev','Sir'],'Other') #나머지 other로 묶어주기
test['identity']=test['identity'].replace(['Capt','Col','Countess','Don','Dr','Jonkheer','Lady','Major','Mlle','Master','Mile','Mme','Ms','Rev','Sir'],'Other') #나머지 other로 묶어주기


In [ ]:
grouped=train['Age'].groupby(train['identity'])#Age를 identity 기준으로 그룹화한 후 평균 계산
grouped.mean()
grouped_test=test['Age'].groupby(test['identity'])#Age를 identity 기준으로 그룹화한 후 평균 계산
grouped_test.mean()

In [ ]:
#그룹별 평균 넣어주기
#train['Age']=train['Age'].fillna(train['Age'].mean())
#test['Age']=test['Age'].fillna(test['Age'].mean())
train['Age']=train['Age'].fillna(grouped.transform('mean'))
test['Age']=test['Age'].fillna(grouped_test.transform('mean'))

In [ ]:
#Nan 값 있는지 확인
train.isnull().sum()
test.isnull().sum()

In [ ]:
#Cabin -> 결측값이 너무 많아 오히려 도움이 안 될 듯 -> 제거
train=train.drop(columns=['PassengerId','Cabin','Name','Ticket','SibSp','Parch'],axis=1)
test=test.drop(columns=['Cabin','Name','Ticket','SibSp','Parch'],axis=1)

In [ ]:
print(train.head())
print(test.head())

In [ ]:
#0과 1로 구성된 더미값 생성 -원핫인코딩
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [ ]:
train_label = train['Survived']
train_data = train.drop('Survived', axis=1)
test_data = test.drop("PassengerId", axis=1).copy()

In [ ]:
import numpy as np
cols= ['Age', 'Fare']

for col in cols:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

In [ ]:
#모델 학습
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

train_data, train_label = shuffle(train_data, train_label,random_state = 5)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression 

model=LogisticRegression()
score=cross_val_score(model,train_data,train_label,scoring='accuracy',cv=3).mean()
score

In [ ]:
def train_and_test(model):
    model.fit(train_data,train_label)
    prediction=model.predict(test_data)
    accuracy=round(model.score(train_data,train_label)*100,2)
    print("Accuracy : ", accuracy, "%")
    return prediction

#LogisticRegression: 주목하는 사건이 발생하는 확률 예측
log_pred = train_and_test(LogisticRegression())


In [ ]:
submission= pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":log_pred})
submission.to_csv('submission.csv', index=False)
submission=pd.read_csv('submission.csv')
submission.head()

In [ ]:
#submission= pd.DataFrame("Id": test.["PassengerId"],"Survived":Y_test)
#submission.to_csv('submission.csv', index=False)